# Structured Outputs : Sorties JSON Garanties avec OpenAI

Dans ce notebook, nous explorons les **Structured Outputs**, une fonctionnalité qui garantit que le modèle retourne des données JSON valides selon un schéma prédéfini.

**Objectifs :**
- Comprendre la différence entre JSON mode et Structured Outputs
- Maîtriser `response_format` avec JSON Schema
- Intégrer Pydantic pour une validation type-safe
- Gérer les erreurs de schéma

**Prérequis :** Notebook 1 (OpenAI Intro)

**Durée estimée :** 55 minutes

In [ ]:
# Installation des dépendances et configuration
%pip install -q openai python-dotenv pydantic

import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional

load_dotenv('../.env')
client = OpenAI()

# Charger le modèle depuis .env ou utiliser gpt-4o par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

def add_additional_properties_false(schema: dict) -> dict:
    """
    Ajoute récursivement additionalProperties: False à tous les objets du schema
    ET s'assure que tous les champs sont dans required (requis par OpenAI).
    
    OpenAI Structured Outputs exige:
    1. additionalProperties: False sur tous les objets
    2. Tous les champs dans 'required' (même ceux avec defaults)
    """
    if not isinstance(schema, dict):
        return schema
    
    # Traiter les définitions $defs d'abord (Pydantic 2.x les génère)
    if "$defs" in schema:
        for def_name, def_schema in schema["$defs"].items():
            add_additional_properties_false(def_schema)
    
    # Si c'est un objet, ajouter additionalProperties: False
    # ET s'assurer que tous les champs sont dans required
    if schema.get("type") == "object":
        schema["additionalProperties"] = False
        # Ajouter tous les champs de properties dans required
        if "properties" in schema:
            all_props = list(schema["properties"].keys())
            schema["required"] = all_props
            # Traiter récursivement les propriétés
            for prop_name, prop_schema in schema["properties"].items():
                add_additional_properties_false(prop_schema)
    
    # Traiter les items d'arrays
    if schema.get("type") == "array" and "items" in schema:
        add_additional_properties_false(schema["items"])
    
    # Traiter anyOf, allOf, oneOf
    for key in ["anyOf", "allOf", "oneOf"]:
        if key in schema:
            for item in schema[key]:
                add_additional_properties_false(item)
    
    return schema

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")

## 1. JSON Mode vs Structured Outputs

OpenAI propose deux approches pour obtenir des sorties JSON :

### JSON Mode
- Garantit une **sortie JSON valide** (pas de syntaxe invalide)
- **NE garantit PAS** la structure exacte du JSON
- Le modèle peut inventer des clés, omettre des champs, etc.
- Utilise `response_format={"type": "json_object"}`

### Structured Outputs
- Garantit une sortie JSON **valide ET conforme au schéma**
- Le modèle respecte strictement le JSON Schema fourni
- Pas de champs manquants, pas de types incorrects
- Utilise `response_format={"type": "json_schema", "json_schema": {...}}`
- Disponible depuis août 2024 (gpt-4o-2024-08-06 et versions ultérieures)

**Cas d'usage :**
- JSON Mode : expérimentation rapide, structure flexible
- Structured Outputs : production, insertion base de données, APIs strictes

In [ ]:
# JSON mode basique (ancienne approche)
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Liste 3 fruits avec leurs couleurs en JSON"}],
    response_format={"type": "json_object"}
)

result = json.loads(response.choices[0].message.content)
print("JSON Mode - Structure non garantie :")
print(json.dumps(result, indent=2, ensure_ascii=False))
print("\nNote : Le modèle a choisi la structure librement (clé 'fruits', sous-objets, noms de clés)")

## 2. Introduction aux Structured Outputs

Les Structured Outputs utilisent **JSON Schema** pour définir précisément la structure attendue.

**Syntaxe :**
```python
response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "nom_du_schema",
        "strict": True,  # Mode strict obligatoire
        "schema": {       # JSON Schema
            "type": "object",
            "properties": {...},
            "required": [...],
            "additionalProperties": False
        }
    }
}
```

**Points clés :**
- `strict: True` est **obligatoire** pour les Structured Outputs
- `additionalProperties: False` empêche les champs non définis
- Tous les champs obligatoires doivent être dans `required`
- Types supportés : `string`, `number`, `integer`, `boolean`, `array`, `object`, `enum`

In [ ]:
# Structured Output avec schéma manuel
schema = {
    "type": "object",
    "properties": {
        "nom": {"type": "string"},
        "ingredients": {"type": "array", "items": {"type": "string"}},
        "temps_minutes": {"type": "integer"},
        "difficulte": {"type": "string", "enum": ["facile", "moyen", "difficile"]}
    },
    "required": ["nom", "ingredients", "temps_minutes", "difficulte"],
    "additionalProperties": False
}

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Donne-moi une recette de crêpes."}],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recette_schema",
            "strict": True,
            "schema": schema
        }
    }
)

recette = json.loads(response.choices[0].message.content)
print("Structured Output - Structure garantie :\n")
print(f"Recette: {recette['nom']}")
print(f"Difficulté: {recette['difficulte']}")
print(f"Temps: {recette['temps_minutes']} minutes")
print(f"Ingrédients: {', '.join(recette['ingredients'])}")

## 3. Intégration avec Pydantic

Écrire des JSON Schemas manuellement est verbeux et source d'erreurs. **Pydantic** simplifie cela :

**Avantages :**
- **Type safety** : Validation automatique des types Python
- **Auto-complétion** : IDE détecte les champs et types
- **Génération automatique** : `model_json_schema()` crée le JSON Schema
- **Validation bidirectionnelle** : `model_validate_json()` vérifie et convertit
- **Documentation** : `Field(description=...)` enrichit le schéma

**Workflow :**
1. Définir les modèles Pydantic avec `BaseModel`
2. Générer le JSON Schema avec `.model_json_schema()`
3. Envoyer à l'API OpenAI
4. Valider la réponse avec `.model_validate_json()`

In [ ]:
# Pydantic avec model_json_schema()
class Ingredient(BaseModel):
    nom: str = Field(description="Nom de l'ingrédient")
    quantite: str = Field(description="Quantité nécessaire")

class RecetteStructuree(BaseModel):
    nom: str = Field(description="Nom de la recette")
    description: str = Field(description="Description courte")
    ingredients: List[Ingredient] = Field(description="Liste des ingrédients")
    etapes: List[str] = Field(description="Étapes de préparation")
    temps_preparation: int = Field(description="Temps en minutes")
    difficulte: str = Field(description="Niveau de difficulté")

# Générer le schema et ajouter additionalProperties: False
schema = RecetteStructuree.model_json_schema()
schema = add_additional_properties_false(schema)

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Donne-moi une recette de mousse au chocolat détaillée."}],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recette_detaillee",
            "strict": True,
            "schema": schema
        }
    }
)

# Validation automatique avec Pydantic
recette = RecetteStructuree.model_validate_json(response.choices[0].message.content)
print(f"Recette: {recette.nom}")
print(f"Temps: {recette.temps_preparation} min | Difficulté: {recette.difficulte}")
print(f"\nDescription:\n{recette.description}")
print("\nIngrédients:")
for ing in recette.ingredients:
    print(f"  - {ing.nom}: {ing.quantite}")
print("\nÉtapes:")
for i, etape in enumerate(recette.etapes, 1):
    print(f"  {i}. {etape}")

## 4. Cas d'Usage Avancés

Les Structured Outputs excellent dans les scénarios nécessitant une **structure de données fiable** :

### Extraction de Données
- **Documents** : CVs, factures, contrats
- **Emails** : Extraction expéditeur, sujet, actions requises
- **Articles** : Métadonnées (titre, auteur, date, mots-clés)

### Génération de Formulaires
- **Questionnaires dynamiques** : Générer des questions avec types de réponse
- **Configurations** : Paramètres applicatifs structurés

### Intégrations API
- **Réponses normalisées** : Format uniforme pour frontend
- **Insertion base de données** : Validation avant INSERT/UPDATE
- **Webhooks** : Payload structuré pour intégrations tierces

### Workflows Multi-Étapes
- **Agents** : Chaque étape retourne une structure validée
- **Pipelines** : Chaînage de transformations type-safe

In [ ]:
# Extraction structurée de CV
# Note: Pour Structured Outputs strict, tous les champs doivent être dans required.
# Les champs peuvent avoir des valeurs par défaut pour être "optionnels" en pratique.

class Experience(BaseModel):
    entreprise: str
    poste: str
    dates: str
    description: str = ""  # Valeur par défaut au lieu d'Optional

class Formation(BaseModel):
    etablissement: str
    diplome: str
    annee: int

class CVStructure(BaseModel):
    nom: str
    email: str = ""  # Valeur par défaut
    telephone: str = ""  # Valeur par défaut
    experiences: List[Experience]
    formations: List[Formation]
    competences: List[str]

texte_cv = """
Jean Dupont - jean.dupont@email.com - 06 12 34 56 78

EXPÉRIENCES:
- Développeur Senior chez TechCorp (2020-2023): Développement d'applications Python
- Développeur Junior chez StartupXYZ (2018-2020): Maintenance de bases de données

FORMATION:
- Master Informatique, Université Paris-Saclay, 2018
- Licence Informatique, Université Lyon 1, 2016

COMPÉTENCES: Python, SQL, Docker, Git, Machine Learning
"""

# Générer le schema et ajouter additionalProperties: False
schema = CVStructure.model_json_schema()
schema = add_additional_properties_false(schema)

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": f"Extrais les informations de ce CV:\n\n{texte_cv}"}],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "cv_extraction",
            "strict": True,
            "schema": schema
        }
    }
)

cv = CVStructure.model_validate_json(response.choices[0].message.content)
print("=== CV STRUCTURÉ ===\n")
print(f"Candidat: {cv.nom}")
print(f"Contact: {cv.email} | {cv.telephone}")
print(f"\nExpériences ({len(cv.experiences)}):")
for exp in cv.experiences:
    print(f"  - {exp.poste} @ {exp.entreprise} ({exp.dates})")
    if exp.description:
        print(f"    -> {exp.description}")
print(f"\nFormations ({len(cv.formations)}):")
for form in cv.formations:
    print(f"  - {form.diplome} ({form.etablissement}, {form.annee})")
print(f"\nCompétences: {', '.join(cv.competences)}")

## 5. Gestion des Erreurs

Même avec Structured Outputs, plusieurs types d'erreurs peuvent survenir :

### Erreurs API
- **Quota dépassé** : Limite de tokens ou requêtes
- **Modèle non supporté** : Structured Outputs requiert gpt-4o-2024-08-06+
- **Schéma invalide** : JSON Schema mal formé ou trop complexe

### Erreurs de Validation Pydantic
- **Type mismatch** : Le modèle retourne un type incorrect (rare avec strict mode)
- **Contraintes non respectées** : `Field(ge=0, le=100)` violé
- **Champs manquants** : Champ requis absent (ne devrait pas arriver)

### Bonnes Pratiques
- **Try/except** : Toujours wrapper les appels API
- **Validation explicite** : Utiliser `model_validate_json()` plutôt que `json.loads()`
- **Logging** : Logger les erreurs pour debug
- **Fallback** : Prévoir une stratégie de secours (retry, valeur par défaut)

In [ ]:
# Gestion robuste des erreurs
from pydantic import ValidationError, conint

def extract_structured(prompt: str, model_class: type[BaseModel], model: str = None):
    """
    Fonction utilitaire pour extraction structurée avec gestion d'erreurs.

    Args:
        prompt: Prompt utilisateur
        model_class: Classe Pydantic pour validation
        model: Modèle OpenAI à utiliser (défaut: DEFAULT_MODEL)

    Returns:
        Instance validée de model_class ou None en cas d'erreur
    """
    if model is None:
        model = DEFAULT_MODEL
    
    try:
        # Générer le schema et ajouter additionalProperties: False (requis par OpenAI)
        schema = model_class.model_json_schema()
        schema = add_additional_properties_false(schema)

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": model_class.__name__.lower(),
                    "strict": True,
                    "schema": schema
                }
            }
        )
        return model_class.model_validate_json(response.choices[0].message.content)
    except ValidationError as e:
        print(f"Erreur de validation Pydantic: {e}")
        return None
    except Exception as e:
        print(f"Erreur API: {e}")
        return None

# Test de la fonction utilitaire
class Evaluation(BaseModel):
    score: conint(ge=0, le=100) = Field(description="Score de 0 à 100")
    commentaire: str = Field(description="Commentaire détaillé")
    points_forts: List[str] = Field(description="Points forts identifiés")
    points_amelioration: List[str] = Field(description="Points à améliorer")

eval_result = extract_structured(
    "Évalue ce texte: 'Python est un langage de programmation populaire.'",
    Evaluation
)

if eval_result:
    print("=== ÉVALUATION STRUCTURÉE ===\n")
    print(f"Score: {eval_result.score}/100")
    print(f"Commentaire: {eval_result.commentaire}")
    print("\nPoints forts:")
    for pf in eval_result.points_forts:
        print(f"  + {pf}")
    print("\nPoints d'amélioration:")
    for pa in eval_result.points_amelioration:
        print(f"  - {pa}")
else:
    print("Échec de l'extraction structurée")

## 6. Exemple Complet : Analyseur de Feedback Produit

Cas d'usage réaliste : analyser des feedbacks clients et les structurer pour analyse.

**Objectifs :**
- Extraire sentiment, catégorie, priorité
- Identifier les fonctionnalités mentionnées
- Détecter les bugs signalés
- Structurer pour insertion en base de données

In [ ]:
# Analyseur de feedback produit
class Bug(BaseModel):
    description: str = Field(description="Description du bug")
    severite: str = Field(description="Sévérité: critique, haute, moyenne, basse")

class FeedbackAnalyse(BaseModel):
    sentiment: str = Field(description="Sentiment général: positif, négatif, neutre, mixte")
    categorie: str = Field(description="Catégorie: feature_request, bug_report, question, praise, complaint")
    priorite: str = Field(description="Priorité: haute, moyenne, basse")
    resume: str = Field(description="Résumé en une phrase")
    fonctionnalites_mentionnees: List[str] = Field(description="Fonctionnalités citées")
    bugs_detectes: List[str] = Field(description="Bugs identifiés")
    suggestions: List[str] = Field(description="Suggestions d'amélioration")
    actions_recommandees: List[str] = Field(description="Actions à entreprendre")

feedback_client = """
J'adore l'application en général, mais le bouton d'export ne fonctionne pas depuis la dernière mise à jour.
J'ai essayé plusieurs fois et rien ne se passe. C'est vraiment frustrant car j'en ai besoin pour mon travail.
Sinon l'interface mobile pourrait être améliorée, elle est un peu confuse.
"""

analyse = extract_structured(
    f"Analyse ce feedback client et structure-le:\n\n{feedback_client}",
    FeedbackAnalyse
)

if analyse:
    print("=== ANALYSE DE FEEDBACK ===\n")
    print(f"Sentiment: {analyse.sentiment}")
    print(f"Catégorie: {analyse.categorie}")
    print(f"Priorité: {analyse.priorite}")
    print(f"\nRésumé:\n{analyse.resume}")
    print(f"\nFonctionnalités mentionnées:")
    for feat in analyse.fonctionnalites_mentionnees:
        print(f"  • {feat}")
    if analyse.bugs_detectes:
        print(f"\nBugs détectés:")
        for bug in analyse.bugs_detectes:
            print(f"  🐛 {bug}")
    if analyse.suggestions:
        print(f"\nSuggestions:")
        for sug in analyse.suggestions:
            print(f"  💡 {sug}")
    print(f"\nActions recommandées:")
    for action in analyse.actions_recommandees:
        print(f"  → {action}")

❌ Erreur API: Error code: 400 - {'error': {'message': "Invalid schema for response_format 'feedbackanalyse': In context=(), 'additionalProperties' is required to be supplied and to be false.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}


## 7. Comparaison des Approches

Récapitulatif des différentes méthodes pour obtenir des sorties structurées :

In [12]:
# Tableau comparatif
print("╔════════════════════════════════════════════════════════════════════════════════════╗")
print("║                      COMPARAISON DES APPROCHES JSON                                ║")
print("╠════════════════════════════════════════════════════════════════════════════════════╣")
print("║ Approche              │ JSON valide │ Structure garantie │ Type-safe │ Complexité ║")
print("╠════════════════════════════════════════════════════════════════════════════════════╣")
print("║ Prompt classique      │      ❌     │         ❌         │     ❌    │   Faible   ║")
print("║ JSON mode             │      ✅     │         ❌         │     ❌    │   Faible   ║")
print("║ JSON Schema manuel    │      ✅     │         ✅         │     ❌    │   Élevée   ║")
print("║ Pydantic + Structured │      ✅     │         ✅         │     ✅    │   Moyenne  ║")
print("╚════════════════════════════════════════════════════════════════════════════════════╝")
print("\nRecommandations :")
print("  • Prototypage rapide → JSON mode")
print("  • Production / APIs → Pydantic + Structured Outputs")
print("  • Insertion BDD → Pydantic + Structured Outputs")
print("  • Workflows complexes → Pydantic + Structured Outputs")

╔════════════════════════════════════════════════════════════════════════════════════╗
║                      COMPARAISON DES APPROCHES JSON                                ║
╠════════════════════════════════════════════════════════════════════════════════════╣
║ Approche              │ JSON valide │ Structure garantie │ Type-safe │ Complexité ║
╠════════════════════════════════════════════════════════════════════════════════════╣
║ Prompt classique      │      ❌     │         ❌         │     ❌    │   Faible   ║
║ JSON mode             │      ✅     │         ❌         │     ❌    │   Faible   ║
║ JSON Schema manuel    │      ✅     │         ✅         │     ❌    │   Élevée   ║
║ Pydantic + Structured │      ✅     │         ✅         │     ✅    │   Moyenne  ║
╚════════════════════════════════════════════════════════════════════════════════════╝

Recommandations :
  • Prototypage rapide → JSON mode
  • Production / APIs → Pydantic + Structured Outputs
  • Insertion BDD → Pydantic + Structur

## Conclusion et Exercices

### Résumé

Les **Structured Outputs** d'OpenAI révolutionnent l'intégration LLM en production :

✅ **Fiabilité** : Garantie de conformité au schéma (plus de parsing fragile)  
✅ **Type Safety** : Pydantic apporte validation et auto-complétion  
✅ **Productivité** : Moins de code de validation manuelle  
✅ **Maintenabilité** : Schémas centralisés et réutilisables  

**Points clés à retenir :**
- `strict: True` est obligatoire pour Structured Outputs
- Pydantic simplifie drastiquement la définition de schémas
- Toujours valider avec `.model_validate_json()` pour type safety
- Gérer les erreurs API et de validation explicitement

---

### Exercices Suggérés

#### Exercice 1 : Extracteur d'Événements (Débutant)
Créer un modèle Pydantic pour extraire des événements d'un texte :
- Titre de l'événement
- Date et heure
- Lieu
- Participants (liste)

Tester avec : *"Réunion d'équipe le 15 mars 2025 à 14h30 dans la salle A. Participants : Alice, Bob, Charlie."*

#### Exercice 2 : Analyseur de Factures (Intermédiaire)
Définir un schéma pour extraire :
- Numéro de facture
- Date d'émission
- Fournisseur (nom, adresse)
- Articles (nom, quantité, prix unitaire, total)
- Montant total TTC

Générer une fausse facture avec GPT et extraire les données.

#### Exercice 3 : Générateur de Quiz (Avancé)
Créer un système qui :
1. Génère un quiz sur un sujet donné (5 questions)
2. Chaque question a 4 choix et une réponse correcte
3. Inclut une explication pour chaque réponse
4. Valide que la réponse correcte est bien dans les choix

Bonus : Ajouter des contraintes Pydantic (ex: `@field_validator`)

---

### Ressources

- [Documentation OpenAI Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs)
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [JSON Schema Reference](https://json-schema.org/understanding-json-schema/)
- [OpenAI Cookbook - Structured Outputs](https://cookbook.openai.com/examples/structured_outputs_intro)

---

**Prochaine étape :** Notebook 4 - Function Calling et Agents